In [30]:
import pandas as pd

In [31]:
df_temps = pd.read_csv('../data/historico_temperaturas.csv', sep=';')
df_prec = pd.read_csv('../data/historico_precipitaciones.csv', sep=';')

In [32]:
print(df_prec.info())
print(df_temps.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   anio    354 non-null    int64 
 1   mes     354 non-null    object
 2   mm      354 non-null    object
 3   dias    354 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 11.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   anio    354 non-null    int64 
 1   mes     354 non-null    object
 2   máxima  354 non-null    object
 3   mínima  354 non-null    object
 4   media   354 non-null    object
dtypes: int64(1), object(4)
memory usage: 14.0+ KB
None


In [33]:
print(df_temps.head())
print(df_prec.head())

   anio      mes máxima mínima  media
0  1991    Enero   28,4   19,8   24,1
1  1991  Febrero   27,8   17,8   22,8
2  1991    Marzo   27,6   18,4     23
3  1991    Abril   22,4   13,6     18
4  1991     Mayo   20,7   12,2  16,45
   anio      mes      mm  dias
0  1991    enero  190,00     7
1  1991  febrero   30,50     6
2  1991    marzo   55,00     8
3  1991    abril  125,60    12
4  1991     mayo   68,40     7


In [34]:
month_map = {
    "enero": "01", "febrero": "02", "marzo": "03", "abril": "04",
    "mayo": "05", "junio": "06", "julio": "07", "agosto": "08",
    "septiembre": "09", "octubre": "10", "noviembre": "11", "diciembre": "12"
}

def create_date_column(df, anio_col="anio", mes_col="mes"):
    df = df.copy()
    df[mes_col] = df[mes_col].str.lower().str.strip()  # Normalize month names
    df["date"] = df[anio_col].astype(str) + "-" + df[mes_col].map(month_map)
    df["date"] = pd.to_datetime(df["date"], format="%Y-%m")  # Convert to datetime
    return df

# Apply the function to both DataFrames
df_prec = create_date_column(df_prec)
df_temps = create_date_column(df_temps)

In [35]:
df_combined = pd.merge(df_prec.drop(columns=["anio", "mes"]),
                       df_temps.drop(columns=["anio", "mes"]),
                       on="date", how="inner")

df_combined.head()

,mm,dias,date,máxima,mínima,media
0,"190,00",7,1991-01-01,"28,4","19,8","24,1"
1,"30,50",6,1991-02-01,"27,8","17,8","22,8"
2,"55,00",8,1991-03-01,"27,6","18,4",23
3,"125,60",12,1991-04-01,"22,4","13,6",18
4,"68,40",7,1991-05-01,"20,7","12,2","16,45"


In [36]:
df_combined = df_combined.rename(columns={
    "mm": "precipitation_mm",
    "dias": "rainy_days",
    "máxima": "max_temp",
    "mínima": "min_temp",
    "media": "avg_temp"
})

df_combined.head()

,precipitation_mm,rainy_days,date,max_temp,min_temp,avg_temp
0,"190,00",7,1991-01-01,"28,4","19,8","24,1"
1,"30,50",6,1991-02-01,"27,8","17,8","22,8"
2,"55,00",8,1991-03-01,"27,6","18,4",23
3,"125,60",12,1991-04-01,"22,4","13,6",18
4,"68,40",7,1991-05-01,"20,7","12,2","16,45"


In [37]:
df_combined.to_csv('historico_clima.csv', index=False)